In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
train_dir = "C:/Users/DELL/Downloads/zdataset/Alzheimer_s Dataset/train"
test_dir = "C:/Users/DELL/Downloads/zdataset/Alzheimer_s Dataset/test"

In [6]:
img_size = (128, 128)
batch_size = 32

datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    subset='training',
    class_mode='categorical'
)
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical'
)

cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

features_train = cnn_model.predict(train_generator)
features_test = cnn_model.predict(test_generator)

cnn_model.save('cnn_feature_extractor.h5')

ann_model = Sequential([
    Input(shape=(features_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(4, activation='softmax') 
])

ann_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

train_labels = np.array(train_generator.labels)
test_labels = np.array(test_generator.labels)
train_labels_one_hot = tf.keras.utils.to_categorical(train_labels, num_classes=4)
test_labels_one_hot = tf.keras.utils.to_categorical(test_labels, num_classes=4)

ann_model.fit(features_train, train_labels_one_hot, validation_data=(features_test, test_labels_one_hot), epochs=10)

predictions = np.argmax(ann_model.predict(features_test), axis=1)
test_accuracy = accuracy_score(test_labels, predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

ann_model.save('ann_model.h5')

def classify_image(img_path):
    cnn_model = tf.keras.models.load_model('cnn_feature_extractor.h5')
    ann_model = tf.keras.models.load_model('ann_model.h5')

    img = image.load_img(img_path, target_size=img_size)  
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) 
    img_array /= 255.0  

    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1) 

    prediction = ann_model.predict(features)
    predicted_class = np.argmax(prediction, axis=1)  
    
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented', 2: 'NonDemented', 3: 'VeryMildDemented'}
    result = class_labels[predicted_class[0]]
    return result

img_path = r"C:/Users/DELL/Downloads/zdataset/Alzheimer_s Dataset/test/NonDemented/32 (7).jpg"
result = classify_image(img_path)
print(f"The image is classified as: {result}")


Found 4098 images belonging to 4 classes.
Found 254 images belonging to 4 classes.


C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


129/129 ━━━━━━━━━━━━━━━━━━━━ 12s 94ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step


Epoch 1/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.4921 - loss: 1.1493 - val_accuracy: 0.5039 - val_loss: 1.0211
Epoch 2/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5098 - loss: 1.0374 - val_accuracy: 0.5039 - val_loss: 1.0211
Epoch 3/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5008 - loss: 1.0441 - val_accuracy: 0.5039 - val_loss: 1.0232
Epoch 4/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5003 - loss: 1.0441 - val_accuracy: 0.5039 - val_loss: 1.0226
Epoch 5/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5033 - loss: 1.0355 - val_accuracy: 0.5039 - val_loss: 1.0236
Epoch 6/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5134 - loss: 1.0165 - val_accuracy: 0.5039 - val_loss: 1.0211
Epoch 7/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4940 - loss: 1.0464 - val_accuracy: 0.5039 - val_loss: 1.0205
Epoch 8/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5070 - loss: 1.0332 - val_accuracy: 0.

Test Accuracy: 50.39%


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
The image is classified as: NonDemented
